In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 3: Spiral Memory ---</h2><p>You come across an experimental new kind of memory stored on an <span title="Good thing we have all these infinite two-dimensional grids lying around!">infinite two-dimensional grid</span>.</p>
<p>Each square on the grid is allocated in a spiral pattern starting at a location marked <code>1</code> and then counting up while spiraling outward. For example, the first few squares are allocated like this:</p>
<pre><code>17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---&gt; ...
</code></pre>
<p>While this is very space-efficient (no squares are skipped), requested data must be carried back to square <code>1</code> (the location of the only access port for this memory system) by programs that can only move up, down, left, or right. They always take the shortest path: the <a href="https://en.wikipedia.org/wiki/Taxicab_geometry">Manhattan Distance</a> between the location of the data and square <code>1</code>.</p>
<p>For example:</p>
<ul>
<li>Data from square <code>1</code> is carried <code>0</code> steps, since it's at the access port.</li>
<li>Data from square <code>12</code> is carried <code>3</code> steps, such as: down, left, left.</li>
<li>Data from square <code>23</code> is carried only <code>2</code> steps: up twice.</li>
<li>Data from square <code>1024</code> must be carried <code>31</code> steps.</li>
</ul>
<p><em>How many steps</em> are required to carry the data from the square identified in your puzzle input all the way to the access port?</p>
</article>


In [2]:
from math import cos, floor, pi, sin, sqrt


square = """
17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---> ...
"""

# https://oeis.org/A214526
# https://oeis.org/A274923
# https://oeis.org/A174344


def a274923(n: int) -> int:
    # a[n_] := a[n] = If[n == 0, 0, a[n-1] - Cos[Mod[Floor[Sqrt[4*(n-1)+1]], 4]* Pi/2]];
    if n <= 1:
        return 0

    a = 0
    for i in range(1, n + 1):
        k = floor(sqrt(4 * (i - 1) + 1)) % 4
        a -= cos(k * pi / 2)
    return round(a)


# fmt: off
a274923_expected = [
    0, 0, 1, 1, 1, 0, -1, -1, -1, -1, 0, 1, 
    2, 2, 2, 2, 2, 1, 0, -1, -2, -2, -2, -2, -2, -2, -1, 
    0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 2, 1, 0, -1, -2, 
    -3, -3, -3, -3, -3, -3, -3, -3, -2, -1, 0, 1, 2, 3, 
    4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 2, 1, 0, -1, -2, -3, 
    -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -3, -2, -1, 0
]
# fmt: on

assert all(a274923(n) == a274923_expected[n] for n in range(len(a274923_expected)))

# fmt: off
a174344_expected = [
    0,1,1,0,-1,-1,-1,0,1,2,2,2,2,1,0,-1,-2,-2,-2,-2,
    -2,-1,0,1,2,3,3,3,3,3,3,2,1,0,-1,-2,-3,-3,-3,-3,
    -3,-3,-3,-2,-1,0,1,2,3,4,4,4,4,4,4,4,4,3,2,1,0,-1,
    -2,-3,-4,-4,-4,-4,-4,-4,-4,-4,-4,-3,-2
 ]
# fmt: on


def a174344(n: int) -> int:
    # a[n_]:=a[n]=If[n==0, 0, a[n-1]+Sin[Mod[Floor[Sqrt[4*(n-1)+1]], 4]*Pi/2]];
    if n < 1:
        return 0

    a = 0
    for i in range(n):
        k = floor(sqrt(4 * i + 1)) % 4
        a += sin(k * pi / 2)
    return round(a)


assert all((a174344(n) == a174344_expected[n]) for n in range(len(a174344_expected)))


# fmt: off
a214526_expected = [
    0,1,2,1,2,1,2,1,2,3,2,3,4,3,2,3,4,3,2,3,4,3,2,3,
    4,5,4,3,4,5,6,5,4,3,4,5,6,5,4,3,4,5,6,5,4,3,4,5,6,
    7,6,5,4,5,6,7,8,7,6,5,4,5,6,7,8,7,6,5,4,5,6,7,8,7,
    6,5,4,5,6,7,8,9,8,7,6,5,6,7,8,9,10,9,8,7,6,5,6,7,
     8,9,10,9,8,7,6,5,6,7,8,9,10,9,8,7,6,5,6,7,8,9,10
]
# fmt: on


def a214526(n: int) -> int:
    # a(n) = abs(A174344(n)) + abs(A274923(n))
    return abs(a174344(n)) + abs(a274923(n))


assert all((a214526(n) == a214526_expected[n]) for n in range(len(a214526_expected)))

tests = [
    {
        "name": "Example 1",
        "square": 1,
        "expected": 0,
    },
    {
        "name": "Example 2",
        "square": 12,
        "expected": 3,
    },
    {
        "name": "Example 3",
        "square": 23,
        "expected": 2,
    },
    {
        "name": "Example 4",
        "square": 1024,
        "expected": 31,
    },
]


def steps(square: int) -> int:
    return a214526(square - 1)


run_tests_params(steps, tests)


Test Example 1 passed, for steps.
Test Example 2 passed, for steps.
Test Example 3 passed, for steps.
Test Example 4 passed, for steps.
Success


In [3]:
print(f"Part I: {steps(325489)}")

Part I: 552


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>552</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>As a stress test on the system, the programs here clear the grid and then store the value <code>1</code> in square <code>1</code>. Then, in the same allocation order as shown above, they store the sum of the values in all adjacent squares, including diagonals.</p>
<p>So, the first few squares' values are chosen as follows:</p>
<ul>
<li>Square <code>1</code> starts with the value <code>1</code>.</li>
<li>Square <code>2</code> has only one adjacent filled square (with value <code>1</code>), so it also stores <code>1</code>.</li>
<li>Square <code>3</code> has both of the above squares as neighbors and stores the sum of their values, <code>2</code>.</li>
<li>Square <code>4</code> has all three of the aforementioned squares as neighbors and stores the sum of their values, <code>4</code>.</li>
<li>Square <code>5</code> only has the first and fourth squares as neighbors, so it gets the value <code>5</code>.</li>
</ul>
<p>Once a square is written, its value does not change. Therefore, the first few squares would receive the following values:</p>
<pre><code>147  142  133  122   59
304    5    4    2   57
330   10    1    1   54
351   11   23   25   26
362  747  806---&gt;   ...
</code></pre>
<p>What is the <em>first value written</em> that is <em>larger</em> than your puzzle input?</p>
</article>

</main>


In [4]:
from functools import cache
from operator import neg

from more_itertools import take


def a274923_a174344(n: int) -> dict[tuple[int, int], int]:
    a = {(0, 0): 1, (0, 1): 2}
    ar, ac = 1, 0
    for i in range(3, n + 1):
        kr = floor(sqrt(4 * (i - 2) + 1)) % 4
        ar += sin(kr * pi / 2)
        kc = floor(sqrt(4 * (i - 2) + 1)) % 4
        ac -= cos(kc * pi / 2)
        a[(-round(ac), round(ar))] = i

    return a


expected_a274923_a174344 = list(zip(map(neg, a274923_expected), a174344_expected))
actual = a274923_a174344(len(expected_a274923_a174344))
assert expected_a274923_a174344 == list(actual.keys())

tests = [
    {
        "name": "Example 1",
        "n": 1,
        "expected": 1,
    },
    {
        "name": "Example 2",
        "n": 2,
        "expected": 1,
    },
    {
        "name": "Example 3",
        "n": 3,
        "expected": 2,
    },
    {
        "name": "Example 4",
        "n": 4,
        "expected": 4,
    },
    {
        "name": "Example 4",
        "n": 5,
        "expected": 5,
    },
]


def sum_of_neighbors(n: int) -> int:
    @cache
    def a141481(n: int) -> int:
        # https://oeis.org/A141481
        if n < 1:
            return 0
        if n == 1:
            return 1

        count = 0
        r, c = n_rc[n]

        for dr, dc in ds:
            n1 = rc_n.get((r + dr, c + dc), inf)
            if not 0 < n1 < n:
                continue
            count += a141481(n1)

        return count

    ds = ((-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1))
    rc_n = a274923_a174344(n)
    n_rc = {v: k for k, v in rc_n.items()}
    return a141481(n)


run_tests_params(sum_of_neighbors, tests)


Test Example 1 passed, for sum_of_neighbors.
Test Example 2 passed, for sum_of_neighbors.
Test Example 3 passed, for sum_of_neighbors.
Test Example 4 passed, for sum_of_neighbors.
Test Example 4 passed, for sum_of_neighbors.
Success


In [5]:
from more_itertools import first


print(
    f"Part II: {first(v for i in range(325489) if (v := sum_of_neighbors(i)) > 325489)}"
)

Part II: 330785


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>330785</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
